In [ ]:
### LSTM-RNN MTM model to predict temporal evolution of: Number of drops (Nd), Interfacial Area (IA), and size distribution (DSD)
### Code adapted for L-L SMX static mixer
### Author: Juan Pablo Valdes
### First commit: Aug, 2023
### Code adapted from: LSTM_MTM by Fuyue Liang, 2023 for stirred vessels
### Department of Chemical Engineering, Imperial College London

# **LSTM timeseries prediction for SMX dispersion performance**

In [1]:
##All imports
import numpy as np
import pandas as pd
import Load_Clean_DF
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns

## *Loading and cleaning up raw data*

In [2]:
cases_3D = ['b03','b06','b09','bi001','bi01','bi1','da001','da01','da1']

cases_MD = ['b06pm','b09pm','bi001pm','bi01pm','da01pm']

cases_clean = ['3drop', 'coarsepm']

Allcases = ['b03','b06','bi001','bi01','da01','da1','b06pm','b09pm','bi001pm',
'bi1','bi01pm','3drop',
'b09','da01pm','da001', 'coarsepm']

<small>Importing rawdata from pre-processed csv files into dataframes</small>

In [3]:
def import_rawdata(case):
    if case == '3drop' or case == 'coarsepm':
        # If true, extract only volume array
        df_Vol = Load_Clean_DF.extract_Vol(case)
    else:
        # If false, extract volume and concentration arrays
        df_Vol = Load_Clean_DF.extract_GVol(case)
    
    # Extract number of drops (Nd) and interfacial area (IntA)
    Nd = Load_Clean_DF.extract_Nd(case)
    IntA = Load_Clean_DF.extract_IA(case)

    return df_Vol, Nd, IntA

<small>Storing all data into a single dictionary to be used later by the LSTM</small>

In [4]:
# Initialize dicts to hold all data extracted from HR sims, before and after post-process
pre_dict = {}
post_dict = {}

# Loop through all cases
for case in Allcases:
    # Extract raw data
    df_Vol, Nd, IntA = import_rawdata(case)
    
    time = Nd['Time']
    n_drops = Nd['Ndrops']
    IA = IntA['IA']
    DSD = df_Vol['Volume']
    
    # Determine if case needs surf. conc. or clean
    if case == '3drop' or case == 'coarsepm':
        G = []  # If true, set G as an empty list
    else:
        G = df_Vol['Gammatilde']  # If false, extract G data
    
    pre_dict[case] = {'Time': time, 'Nd': n_drops, 'IA': IA, 'Vol': DSD, 'G': G}
    
    # Initialize an empty post-process dict per case
    post_dict[case] = {}



### Starting first with Nd and IA

<small>Scale and normalise data to be handled for LSTM</small>

In [5]:
# Import MinMaxScaler from sklearn for scaling and normalization
from sklearn.preprocessing import MinMaxScaler

# List of columns to be normalized
norm_columns = ['Nd', 'IA']

for case in Allcases:
    norm_data_case = pre_dict[case]

    # Loop through each column to be normalized
    for column in norm_columns:
        norm_data = norm_data_case[column].values.astype('float64')
        
        # Reshape the data to be compatible with the scaler
        norm_data = norm_data.reshape(-1, 1)
        
        # Create a MinMaxScaler and fit it to the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler = scaler.fit(norm_data)
        
        # Transform and store the normalized data in the post_data dict.
        post_dict[case][column] = scaler.transform(norm_data).astype('float32')

<small>Stacking all data into a numpy array and reshaping it as : (timesteps, num_cases, num_features)</small>

In [26]:
array = []
### All cases must have the same number of data points for them to be used in LSTM

min_length = min(len(data['IA']) for data in post_dict.values())

# Iterate through each case
for case, features in post_dict.items():
    # Extract the 'Nd' and 'IA' data for the current case
    # Cases are truncated with the min length in order to be stacked as a nparray
    Nd_data = features['Nd'][:min_length]
    IA_data = features['IA'][:min_length]

    
    # Combine 'Nd_data' and 'IA_data' into a single numpy array
    combined_data = np.column_stack((Nd_data, IA_data))
    
    # Append the combined data to the data_list
    array.append(combined_data)

# Stack the data_list along a new axis to create the final numpy array
mydata = np.stack(array, axis=1)
print('(time_step, num_case, num_feature)=', mydata.shape)

(time_step, num_case, num_feature)= (105, 16, 2)


In [ ]:
### looping over the number of features (Nd and IA)

#Plot setup
rc('text', usetex=True)
custom_font = {'family': 'serif', 'serif': ['Computer Modern Roman']}
rc('font', **custom_font)
plt.figure(dpi=150)

features = ['Number of drops', 'Interfacial Area']
colors = sns.color_palette("husl", len(Allcases))

for i in range(mydata.shape[-1]):
    plt.figure(figsize=(10, 6))
    plt.title(f'{features[i]}')
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[i]}')

    ### Feature 1 and 2 being Nd and IA, respectively
    for case, idx in zip(Allcases, range(len(Allcases))):
        plt.plot(mydata[:,idx,i], label = f'{str(case)}',color=colors[idx % len(colors)])
        plt.legend()


plt.tight_layout()
plt.show()

### Smoothing the processed data

In [ ]:
from scipy.signal import savgol_filter
from statsmodels.nonparametric.smoothers_lowess import lowess

def smoothing(data, method, window_size=None, poly_order=None, lowess_frac = None):
    '''
    Input array : 2D array per feature, with shape (times, cases)
    Three methods to smooth:
    
    'moveavg': requires window_size
    'savgol': requires window_size, poly_order
    'lowess': requires lowess_frac
    '''

    ## rolling window averaging method
    if method == 'moveavg':
        if window_size is None:
            raise ValueError('Window size required')
        smoothed_data = pd.DataFrame(data).rolling(window_size, axis = 0).mean()
        smoothed_data.fillna(pd.DataFrame(data),inplace=True)
    ## SavGol filter based on fitting least-squares polynomial to a window of data points
    elif method == 'savgol':
        if window_size is None or poly_order is None:
            raise ValueError('Mising input arguments: Windowsize/polyorder')
        smoothed_data = np.apply_along_axis(
                    lambda col: savgol_filter(col, window_size, poly_order),
                    axis = 0, arr=data)
    ## Locally Weighted Scatterplot Smoothing, locally fitting linear regressions
    elif method == 'lowess':
        if lowess_frac is None:
            raise ValueError('Lowess fraction required')
        smoothed_data = np.apply_along_axis(
                    lambda col: lowess(col,np.arange(len(col)),frac = lowess_frac,return_sorted=False),
                    axis = 0, arr = data)
    else:
        raise ValueError('Unsupported smoothing method')
    
    
    fig, ax = plt.subplots(1,2, figsize=(15,5))

    for case, idx in zip(Allcases, range(len(Allcases))):
        ax[0].plot(data[:,idx], label = f'{str(case)}',color=colors[idx % len(colors)])
        ax[0].set_title('Nd before')
        ax[0].set_xlabel('Time steps')
        ax[0].set_ylabel('Scaled Nd')
        
        ax[1].plot(smoothed_data[:,idx], label = f'{str(case)}',color=colors[idx % len(colors)])
        ax[1].set_title('Nd after')
        ax[1].set_xlabel('Time steps')
        ax[1].set_ylabel('Smoothed Nd')
    
    fig.suptitle(f'Smoothing method: {method}', fontsize=18)

    ax[0].legend()
    ax[1].legend()
    plt.show()

    return smoothed_data

In [ ]:
data = mydata[:,:,0] # 0 for drop number and 1 for IA
#smoothed_data = smoothing(data,'lowess',lowess_frac=0.06)
smoothed_data = smoothing(data,'savgol',window_size=5,poly_order=3)
#smoothed_data = smoothing(data,'moveavg',window_size=5)

## *Train, validation and test data set splitting*

In [ ]:
### split by case
def cases_split(df, train_frac, test_frac, cases):
    '''
    input shape: (times, cases, features)
    
    return train, val data and cases
    '''
    train_size = int(df.shape[1]*train_frac)

    val_size = int(df.shape[1]*(1-test_frac-train_frac))
    
    train, val, test = df[:, :train_size, :], df[:, train_size:(train_size+val_size), :], df[:,(train_size+val_size):,:]
    print(f'number of train, val and test cases: {train.shape[1]}, {val.shape[1]}, {test.shape[1]}')
    
    train_cases, val_cases , test_cases = cases[:train_size], cases[train_size:(train_size+val_size)], cases[(train_size+val_size):]
    print(f'training cases: {train_cases}, validation cases: {val_cases}, test cases: {test_cases}')
        
    return train, val, test, train_cases, val_cases, test_cases

In [ ]:
train_frac = 0.5625
val_frac = 0.1875
test_frac = 0.25

## Replacing noisy Nd data with smoothed data
mydata[:,:,0] = smoothed_data

f_data = mydata

train, val, test, train_cases, val_cases, test_cases = cases_split(f_data, train_frac=train_frac, test_frac = test_frac, cases=Allcases)

print(f'train, val, test shapes: {train.shape}, {val.shape}, {test.shape}')


In [ ]:
c1 = sns.color_palette("Set1", len(Allcases))
c2 = sns.color_palette("Set2", len(Allcases))
c3 = sns.color_palette("Set3", len(Allcases))

# Training cases
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
for i in range(mydata.shape[-1]):
    for case, idx in zip(train_cases, range(len(train_cases))):
        ax[i].plot(train[:,idx,i],label = f'{str(case)}',color=c1[idx % len(colors)])
        ax[i].set_title(f'Training: {features[i]}')
        ax[i].set_xlabel('Time steps')
        ax[i].set_ylabel(f'Scaled {features[i]}')
        ax[i].legend()

# Validation cases
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
for i in range(mydata.shape[-1]):
    for case, idx in zip(val_cases, range(len(val_cases))):
        ax[i].plot(val[:,idx,i],label = f'{str(case)}',color=c2[idx % len(colors)])
        ax[i].set_title(f'Validation: {features[i]}')
        ax[i].set_xlabel('Time steps')
        ax[i].set_ylabel(f'Scaled {features[i]}')
        ax[i].legend()

# Test cases
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
for i in range(mydata.shape[-1]):
    for case, idx in zip(test_cases, range(len(test_cases))):
        ax[i].plot(test[:,idx,i],label = f'{str(case)}',color=c3[idx % len(colors)])
        ax[i].set_title(f'Test: {features[i]}')
        ax[i].set_xlabel('Time steps')
        ax[i].set_ylabel(f'Scaled {features[i]}')
        ax[i].legend()

plt.show()

## *Sampling prediction dataset (windowing)*

In [ ]:
import torch

def window_data(df, steps_in, stride, steps_out):
    '''
    
    df: with shape (times, cases, features)
    stride: the step size between consecutive windows
    pred_times:(<window_size) predicted future times from current window
    window size: Encompasses both steps_in and steps_out, referring to input seq and prediction seq
    
    lookback period = window_size - steps_out
     
    '''
    window_size = steps_in + steps_out
    casebatch_lens = [] # List to contain the number of rows/windows per case used for input-->prediction based on the steps_in - steps_out parameters
                        # Can be calculated as: len(timesteps)-window_size+1
    X, y = [], []

    for i in range(df.shape[1]): # looping for each case, df shape of (times, cases, features)
        df_case = df[:,i,:] # df per case
        for j in range(0, len(df_case)-window_size+1, stride): # Looping over timesteps based on the window size
            wd_data = df_case[j:j+window_size] # window with times: steps_in + steps_out
            X.append(wd_data[:-steps_out]) #input values, steps_in
            y.append(wd_data[-steps_out:]) #training/ prediction values, steps_out
        casebatch_lens.append(len(X)) # appending casebatch length per case 

    ## number of windows/rows with size (steps_in) per case, used to later plot per case
    print(casebatch_lens)
    
    return torch.tensor(X), torch.tensor(y), np.array(casebatch_lens)

    


In [ ]:
steps_in, steps_out = 36, 20
wid_size = steps_in + steps_out
stride = 1

### Creating the tensors (sampling) after splitting train and validation data sets
X_train, y_train, train_casebatch = window_data(train, steps_in=steps_in, stride=stride, steps_out=steps_out)
X_val, y_val, val_casebatch = window_data(val, steps_in=steps_in, stride=stride, steps_out=steps_out)

# Check the shape of the output tensor
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
## output shape should be (windows among all cases: 
# number of windows/rows per case X number of cases, times per 
# input/prediction row: steps_in/steps_out, features per case)


# LSTM MTM (Many-to-Many):

An LSTM unit generates three main outputs:

1. <sub>**Cell state**: This could be thought of as long-term memory, representing the aggregate information of all previous LSTM units (timesteps). It is selectively remembered or forgotten at each LSTM unit.</sub>

2. <sub>**Hidden state**: This is often thought of as short-term memory. It characterizes the most recent timestep, influenced by the cell state and the input, and used to modify the cell state. Here we can consider 'neurons' as in a regular NN through the hidden size, which determines the size of the matrix of weights used to compute the hidden state. This is independent of the number of features but it modifies the size of the matrix of input weights accordingly. </sub>

3. <sub>**Output**: The output is derived directly from the hidden state and represents the final prediction after being properly decoded.</sub>


## 1) Direct Multi-step DMS approach

- <sub> In this approach, we consider a window formatted as steps_in (input) and steps_out (prediction) and we roll the window forward based on the stride. The number of rolls/rows/windows depends on the number of timesteps and the size of the row selected (steps_in and steps_out). 

- <sub> The first LSTM unit receives as input a batch containing the first value of each window, depending on the batch size hyperparameter. After calculations are done, the next LSTM unit receives the previous cell state, hidden state and as input the second value of each window in the batch and continues accordingly.

- <sub> In this approach, we use the last LSTM unit's output (hidden state) and pass it through a dense/linear layer to make predictions for all future time steps simultaneously. We force this multi output scenario. </sub>

<sub>The LSTM output layer produces multiple outputs, allowing it to be trained to directly predict multiple future time points in one pass.</sub>



In [ ]:
import torch.nn as nn

### LSTM model class
class LSTM_DMS(nn.Module):

    ## class constructor
    def __init__(self, input_size, hidden_size, output_size, pred_steps,
                 l1_lambda=0.0, l2_lambda=0.0):
        # calling the constructor of the parent class nn.Module to properly intialize this class
        super(LSTM_DMS,self).__init__()
        self.hidden_size = hidden_size
        self.pred_steps = pred_steps # prediction steps = steps_out
        self.output_size = output_size
        ## LSTM unit instance from parent class
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True) 
        # Linear layer instance from parent class, for multi-step predictions
        self.linear = nn.Linear(hidden_size, output_size * pred_steps)

        # Relevance markers for L1 and L2 regularizations
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    ### forwards pass: How input data will be processed by the network layers
    def forward(self, input):

        # No initialisation for hidden or cell states h0, c0. Could be initialised for a given device (e.g., GPU use through .to(x.device))
        # Inputting data (x) into the LSTM sequence and reading the output per unit and as a whole
        lstm_output, _ = self.lstm(input)#,(h0,c0))
        
        # Get the last hidden state from the LSTM sequence
        last_output = lstm_output[:, -1, :]
        
        # Input the last output from the LSTM into the dense linear layer, where we obtain the multi-output
        multi_step_output = self.linear(last_output)
        
        # Reshape the output to get predictions for multiple future time steps
        multi_step_output = multi_step_output.view(-1, self.pred_steps, self.output_size)

        return multi_step_output
    
    ### Regularization functions to prevent overfitting
    #L1 (lasso) encourages sparse weights
    def l1_regularization_loss(self):
        if self.training:
            l1_loss = 0.0
            for param in self.parameters():
                l1_loss += torch.sum(torch.abs(param))
            return self.l1_lambda * l1_loss
        else:
            return 0

    #L2 (Ridge) encourages small weights
    def l2_regularization_loss(self):
        if self.training:
            l2_loss = 0.0
            for param in self.parameters():
                l2_loss += torch.sum(param ** 2)
            return 0.5 * self.l2_lambda * l2_loss
        else:
            return 0

## LSTM training

<small>Two common approaches for loss calculation in MTM problems:

-__Sequence to Sequence loss__: Prediction sequence output must be the same length as the true future (target) sequence. Loss functions like Cross-Entropy Loss can be used to compare predicted and true future sequence (target).

-__Element-Wise Loss__: Separate loss function between predicted sequence and true future (target) sequence for each individual time step. Final loss value is calculated as the average of all losses. Common functions MSE and MAE. We implement this one here.  <small>

### Customizing the loss function

<small> Adding a penalty term to avoid negative predictions, as features are always positive. <small>

In [ ]:
import torch.nn as nn

class custom_loss(nn.Module):
    # constructor and super from the parent nn.Module class
    def __init__(self, penalty_weight):
        super().__init__()
        self.penalty_weight = penalty_weight
        
    def forward(self, prediction, target):
        # mse as base loss function
        mse_loss = nn.MSELoss()(prediction, target)
        
        # penalise negative prediction
        '''
        -prediction: negates all the values in prediction tensor
        torch.relu: set all the now negative values (i.e., initially positive) as zero and postive (previously negative) values unchanged
        torch.mean: average the penalty for all the previously negative predictions
        '''
        penalty = torch.mean(torch.relu(-prediction))
        
        custom_loss = mse_loss + self.penalty_weight * penalty
        
        return custom_loss 

##### Hyperparameter, optimizer, loss fun, and data loader setup: DMS

In [ ]:
import torch.optim as optim
import torch.utils.data as data
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define hyperparameters
input_size = X_train.shape[-1]  # Number of features in the input tensor
hidden_size = 128  # Number of hidden units in the LSTM cell, determines how many weights will be used in the hidden state calculations
num_layers = 1 # Number of LSTM layers per unit
output_size = y_train.shape[-1]  # Number of output features, same as input in this case
pred_steps = steps_out # Number of future steps to predict
batch_size = 10 # How many windows are being processed per pass through the LSTM
learning_rate = 0.01
num_epochs = 3000
check_epochs = 100

# LSTM model instance
model = LSTM_DMS(input_size, hidden_size, output_size, pred_steps,
                    l1_lambda=0.00, l2_lambda=0.00) 

# customize loss function 
penalty_weight = 10
loss_fn = custom_loss(penalty_weight)

loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
optimizer = optim.Adam(model.parameters(), lr = learning_rate) # optimizer to estimate weights and biases (backpropagation)
                                           # requires_grad=True

# Learning rate scheduler, set on min mode to decrease by factor when validation loss stops decreasing                                       
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

### Training DMS model

<sub> Early stopping feature to avoid overfitting and general training routine

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, model_name, patience=5, verbose=False, delta=0.0001):
        """
        Args:
            patience (int): How long to wait after last improvement in the monitored metric.
                            Default: 5
            verbose (bool): If True, print a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored metric to be considered an improvement.
                            Default: 0.0001
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.name = model_name

    def __call__(self, val_loss, model):
        """
        Args:
            val_loss (float): Validation loss to be monitored for improvement.
            model (nn.Module): Model to be saved if the monitored metric improves.
        """
        score = -val_loss  # Assuming lower validation loss is better.

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """
        Saves model when validation loss decreases.
        """
        if self.verbose:
            print(f'Validation loss decreased ({self.best_score:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), f'{self.name}_trained_model.pt')  # Save the model's state_dict.


#### Training function for DMS

In [ ]:
def training_DMS(model, optimizer, loss_fn, loader, scheduler,
                  num_epochs, check_epochs, 
                  X_val, y_val, saveas):
    with open(str(saveas)+'.txt', 'w') as f:
        print(model, file=f)

        ### Early stopping feature to avoid overfitting during training, monitoring a minimum improvement threshold
        early_stopping = EarlyStopping('DMS',patience=10, verbose=True)

        for epoch in range(num_epochs): #looping through epochs
            model.train() #set the model to train mode -- informing features to behave accordingly for training
            
            first_iteration = True
            for X_batch, y_batch in loader:
                
                optimizer.zero_grad() # setting gradients to zero to start a new run on weight optimisation (clear accumulated from previous batch)

                # Forward pass
                y_pred = model(X_batch)

                # Calculate loss
                loss = loss_fn(y_pred, y_batch)

                # Calculate L1 and L2 regularization terms
                l1_regularization = model.l1_regularization_loss()
                l2_regularization = model.l2_regularization_loss()

                # Add regularization terms to the loss
                loss += l1_regularization + l2_regularization

                # Backpropagation and parameter update
                loss.backward() # calculating the gradient of the loss with respect to the model's parameters (weights and biases)
                                # it acculmulates the gradients each time we go through the nested loop

                optimizer.step() # updating parameters to minimize the loss function

                # Check the shapes in the first iteration of the first epoch
                if epoch == 0 and first_iteration:
                    print('Input shape:', X_batch.shape)
                    print('Output shape:', y_pred.shape)
                    first_iteration = False

            # Validation at each check epoch batch
            if epoch % check_epochs != 0:
                continue

            model.eval() # set the model to evaluation form

            with torch.no_grad(): #predictions performed with no gradient calculations
                y_pred_train = model(X_train)
                y_pred_val = model(X_val)

                t_rmse = loss_fn(y_pred_train, y_train)
                v_rmse = loss_fn(y_pred_val, y_val)

                print('Epoch %d : train RMSE  %.4f, val RMSE %.4f ' % (epoch, t_rmse, v_rmse), file=f)
                print('Epoch %d : train RMSE  %.4f, val RMSE %.4f ' % (epoch, t_rmse, v_rmse))
                
            ## Learning rate scheduler step
            scheduler.step(v_rmse)

            ## early stopping check to avoid overfitting
            early_stopping(v_rmse, model)

            if early_stopping.early_stop:
                print('Early stopping')
                break

    ## Load the last best model before training degrades         
    model.load_state_dict(torch.load('DMS_trained_model.pt'))

In [ ]:
training_DMS(model=model, optimizer=optimizer, loss_fn=loss_fn, loader=loader, scheduler=scheduler, 
             num_epochs=num_epochs, check_epochs=check_epochs,
             X_val=X_val, y_val=y_val,
             saveas='DMS_out')

<sub> Computing predictions with trained model from above.

In [ ]:
### Carry out prediction with final trained model
model.load_state_dict(torch.load('checkpoint.pt'))
model.eval()
with torch.no_grad():

    y_pred_train = model(X_train)
    y_pred_val = model(X_val)
        
#     y_pred_test = model(X_test)
    print(y_pred_train.shape, y_pred_val.shape)

In [ ]:
num_features = input_size #Nd and IA
num_cases = len(train_cases)
colors = sns.color_palette("coolwarm", num_cases)

# Loop over features
for f_idx in range(num_features):
    plt.figure(figsize=(12,6))
    s_idx = -1
    
    for seq, case in zip(range(num_cases), train_cases):
        # Target plots, true data from CFD
        p = plt.plot(train[:, seq, f_idx], label=f'Target {str(case)}', color=colors[seq % len(colors)], linewidth = 2) # train has shape [times,cases,features]
        
        # Train predicted values
        if seq == 0:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[:train_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Train Predicted {str(case)}',linewidth = 4)
        else:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[train_casebatch[seq-1]:train_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Train Predicted {str(case)}', linewidth = 4)
            
    if f_idx == 1:
        plt.ylim(0.6, 1.1)


    plt.legend()
    plt.xlim(40, 105)
    plt.title(f'Training Prediction for {features[f_idx]}')
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[f_idx]}')

In [ ]:
num_features = input_size #Nd and IA
num_cases = len(val_cases)
colors = sns.color_palette("husl", num_cases)

# Loop over features
for f_idx in range(num_features):
    plt.figure(figsize=(12,6))
    s_idx = -1
    
    for seq, case in zip(range(num_cases), val_cases):
        # Target plots, true data from CFD
        p = plt.plot(train[:, seq, f_idx], label=f'Target {str(case)}', color=colors[seq % len(colors)],linewidth = 2) # train has shape [times,cases,features]
        
        # Train predicted values
        if seq == 0:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[:val_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Val Predicted {str(case)}',linewidth = 4)
        else:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[val_casebatch[seq-1]:val_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Val Predicted {str(case)}',linewidth = 4)
    if f_idx == 1:
            plt.ylim(0.6, 1.1)


    plt.legend()
    plt.xlim(40, 105)
    plt.title(f'Validation prediction for {features[f_idx]}')
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[f_idx]}')

In [ ]:
# Function to predict future values using rollout with the LSTM model
def rollout(model, input_seq, future_steps):
    
    ## setting to eval mode and dropping gradient calculation for prediction
    model.load_state_dict(torch.load('DMS_trained_model.pt'))
    model.eval()
    with torch.no_grad():
        
        reshaped_input = np.transpose(input_seq, (1,0,2)) #reshaping to case,inputdata,features

        gen_seq = torch.tensor(reshaped_input).clone().detach() #detaching tensor from computational graph so gradients are not required

        ### how many predicted windows will be calculated based on the input_steps
        num_forwards = int(future_steps / steps_out) + 1
        print(f'prediction iterates for {num_forwards} times.')

        ## window predicton
        for i in range(num_forwards):

            output = model(gen_seq) #LSTM pass

            gen_seq = torch.cat((torch.tensor(gen_seq), torch.tensor(output)), dim=1) # concatenates the predicted outputs to the initial input sequence
            # This new sequence is fed as the new input tensor and used for the next set of output predictions.               

    return gen_seq

In [ ]:
input_seq = test[:steps_in,:,:]
future_steps = test.shape[0] - steps_in
rollout_seq = rollout(model, input_seq, future_steps)
print(input_seq.shape)
print(rollout_seq.shape)

In [ ]:
from sklearn.metrics import r2_score

colors = sns.color_palette("cubehelix", len(test_cases))

for f_idx in range(input_size):
    plt.figure()
    for i, case in enumerate(test_cases):
        r2 = r2_score(test[:,i,f_idx],rollout_seq[i,:,f_idx][:test.shape[0]])

        p = plt.plot(test[:,i,f_idx], label=f'Target {case}, r2:{r2:.4f}',color = colors[i % len(colors)],linewidth = 2)
        plt.plot(rollout_seq[i,:,f_idx],c=p[0].get_color(), linestyle=':',label=r'Prediction',linewidth = 4)
    
    plt.legend()
    plt.title(f'Prediction for: {features[f_idx]}')
    plt.xlim(35, 105)
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[f_idx]}')

### 2) Sequence-to-Sequence S2S approach
<small> - __Sequence-to-Sequence__: Use the LSTM's output for each time step and feed it back as input for the subsequent time step. In this way, LSTM generates predictions step-by-step for each future time step.

This approach consists of two LSTMs, an encoder and a decoder:

- The **encoder LSTM** processes the input sequence an generates an internal (encoded) representation (hidden state), summarizing the input information.
- The **decoder LSTM** uses the encoded state (final hidden state) as its initial hidden state and then generates an output sequence step by step.

We consider the same windowing data-processing step used prior


In [ ]:
### LSTM ENCODER/DECODER AND PREDICTION CLASSES
import torch.nn as nn

class LSTM_encoder(nn.Module):

    # Same as LSTM DMS constructor but with no pred_steps or linear layer as encoder feeds decoder LSTM through the hidden states
    def __init__(self,input_size,hidden_size, num_layers=1):
        super(LSTM_encoder,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, 
                            batch_first=True)
        
    # Take the input sequences and output the hidden states for the LSTM decoder section
    def forward(self, encoder_input):
        ''' 
        return encoder_hidden_states: outputs the last time hidden and cell state to be fed into the LSTM decoder
        
        input shape: (batch_size, input steps/input window, input_size=num_features)
        output shape: (input_size=num_features, hidden_size)
        '''
        _, (h_n_encoder,c_n_encoder) = self.lstm(encoder_input) #ignoring output (hidden states) for all times and only saving a tuple with the last timestep cell and hidden state
        
        return (h_n_encoder,c_n_encoder)

class LSTM_decoder(nn.Module):

    ## Same constructor as DMS as now we are decoding the final LSTM cell through a linear layer to generate the final output 
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTM_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, 
                            batch_first=True)
        
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, decoder_input, encoder_states):
        '''
        return 
        lstm_output: returns decoded hidden states as output for all times 
        
        input shape: (batch_size, 1, input_size=num_features) the last time step
        output shape: (batch_size, input_size=num_features)
        '''

        # LSTM cell is initialized with the encoder cell and hidden states
                # Input tensor is unsqueezed to introduce an additional dimension in axis = 1 to perform LSTM calculations normally for 1 step
        lstm_output, _ = self.lstm(decoder_input.unsqueeze(1), encoder_states) #Similar to DMS, output is saved, representing all hidden states per timestep
        
        ## output tensor is squeezed, removing the aritificial time dimension in axis = 1, as it will be looped during prediction for each time and appended to a 3D tensor.
        output = self.linear(lstm_output.squeeze(1))
        
        return output
    
class LSTM_S2S(nn.Module):
    ''' Double LSTM Encoder-decoder architecture to make predictions '''

    #Constructing the encoder decoder LSTM architecture
    def __init__(self, input_size, hidden_size, output_size, pred_steps):
        super(LSTM_S2S,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.pred_steps = pred_steps #steps out = output window
        
        self.encoder = LSTM_encoder(input_size=input_size, hidden_size=hidden_size)
        self.decoder = LSTM_decoder(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

    def forward(self,input_tensor):
        '''
        input_tensor: shape (batch_size, input steps = input window, input_size=num_features)
        pred_steps: number of time steps to predict
        return np_outputs: array containing predictions
        '''
                
        # encode input_tensor
        encoder_states = self.encoder(input_tensor)

        # initialize output tensor for prediction
        outputs = torch.zeros(input_tensor.shape[0], self.pred_steps, input_tensor.shape[2]) #shape = batch_size, steps_out, num_features


        # decode input_tensor
        decoder_input = input_tensor[:,-1,:] # Taking last value in the window/sequence
        decoder_input_states = encoder_states

        # predictions carried out on the decoder for each time in the output window = steps_out
        for t in range(self.pred_steps):
            decoder_output = self.decoder(decoder_input,decoder_input_states)
            outputs[:,t,:] = decoder_output
            # prediction done recursively
            decoder_input = decoder_output

        np_outputs = outputs.detach().numpy() ## detaching from gradient requirements during prediction

        return torch.from_numpy(np_outputs)


## LSTM Training: S2S

<sub> We keep using the same early stopping and custom loss functions defined for DMS

##### Hyperparameter, optimizer, loss fun, and data loader setup: S2S

In [ ]:
import numpy as np
import random
import torch.optim as optim
import torch.utils.data as data
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define hyperparameters
input_size = X_train.shape[-1]  # Number of features in the input tensor
hidden_size = 128  # Number of hidden units in the LSTM cell, determines how many weights will be used in the hidden state calculations
num_layers = 1 # Number of LSTM layers per unit
output_size = y_train.shape[-1]  # Number of output features, same as input in this case
pred_steps = steps_out # Number of future steps to predict
batch_size = 10 # How many windows are being processed per pass through the LSTM
learning_rate = 0.001
num_epochs = 3000
check_epochs = 100

# For teacher forcing
tf_ratio = 0.1 
dynamic_tf = False

# Pick one type of model #
model = LSTM_S2S(input_size, hidden_size, output_size, pred_steps) # create out NN


# customize loss function 
penalty_weight = 10
loss_fn = custom_loss(penalty_weight)

loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
optimizer = optim.Adam(model.parameters(), lr = learning_rate) # optimizer to estimate weights and biases (backpropagation)
                                           # requires_grad=True

# Learning rate scheduler, set on min mode to decrease by factor when validation loss stops decreasing                                       
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

<sub> Encoding-decoding LSTM architectures are not coded here, as they have been defined already in the S2S class. Hence, the training here only calls the forward functions from each step as required.

In [ ]:
def training_S2S(model, optimizer, loss_fn, loader,scheduler, 
                 num_epochs, check_epochs, pred_steps,
                 training_prediction, tf_ratio, dynamic_tf,
                 X_val, y_val,
                 saveas):
    ''' 
    training_prediction: ('recursive'/'teacher_forcing'/'mixed')
    tf_ratio: float[0,1] 
                relevance on teacher forcing when training_prediction = 'teacher_forcing'.
                For each batch, a random number is generated. 
                If the number is less than tf_ratio, tf is used; otherwise, prediction is done recursively.
                If tf_ratio = 1, only tf is used.
    dynamic_tf: (True/False)
                dynamic teacher forcing reduces the amount of teacher forcing for each epoch
    
    return loss: array of loss function for each epoch
    '''

    # save the training model
    with open(str(saveas)+'.txt', 'w') as f:
        print(model, file=f)

        ### Early stopping feature to avoid overfitting during training, monitoring a minimum improvement threshold
        early_stop = EarlyStopping('S2S',patience=10, verbose=True)

        for epoch in range(num_epochs): #looping through training epochs
            
            model.train() #setting model to training function to deactivate regularization and other training features
            first_iteration = True

            for X_batch, y_batch in loader:

                # initializing output tensor
                outputs = torch.zeros(X_batch.shape[0], pred_steps, X_batch.shape[2]) #shape = (batch_size,steps_out,num_features)

                #reset gradients from previous training step
                optimizer.zero_grad()

                #going through the LSTM encoder layer: return hidden and cell states
                encoder_states = model.encoder(X_batch)

                # decoder starting with teacher forcing: input set as last timestep from input batch
                decoder_input = X_batch[:,-1,:] # in shape of (batch_size, input_size = num_features)
                decoder_input_states = encoder_states

                #Considering variations in training methods per batch
                if training_prediction == 'recursive':
                        
                    # recursive prediction: predicted output is fed
                        for t in range(pred_steps):
                            decoder_output = model.decoder(decoder_input, decoder_input_states)
                            outputs[:,t,:] = decoder_output
                            decoder_input = decoder_output


                if training_prediction == 'teacher_forcing':
                        
                    # predict using teacher forcing: target is fed
                        if random.random() < tf_ratio:
                            for t in range(pred_steps):
                                decoder_output = model.decoder(decoder_input, decoder_input_states)
                                outputs[:,t,:] = decoder_output
                                decoder_input = y_batch[:,t,:] # target fed from y_batch in shape of (batch_size, input_size = num_features)
                        # predict recursively
                        else:
                            for t in range(pred_steps):
                                decoder_output = model.decoder(decoder_input, decoder_input_states)
                                outputs[:,t,:] = decoder_output
                                decoder_input = decoder_output


                if training_prediction == 'mixed':

                    # both types of training methods used in the same batch, alternating stochastically based on tf_ratio
                    for t in range(pred_steps):
                        decoder_output = model.decoder(decoder_input, decoder_input_states)
                        outputs[:,t,:] = decoder_output

                        ## Teaching method chosen per timestep within the given batch
                        # teacher forcing
                        if random.random() < tf_ratio:
                            decoder_input = y_batch[:,t,:]
                        # recursive:
                        else:
                            decoder_input = decoder_output

                loss = loss_fn(outputs,y_batch)

                # Backpropagation and parameter update
                loss.backward() # calculating the gradient of the loss with respect to the model's parameters (weights and biases)
                                # it acculmulates the gradients each time we go through the nested loop

                optimizer.step() # updating parameters to minimize the loss function

                # Check the shapes in the first iteration of the first epoch
                if epoch == 0 and first_iteration:
                    print('Input shape:', X_batch.shape)
                    print('Output shape:', outputs.shape)
                    first_iteration = False
                
            # dynamic teacher forcing
            if dynamic_tf and tf_ratio > 0:
                tf_ratio = tf_ratio - 0.02 ## if dynamic tf active, the amount of teacher forcing is reduced per epoch

                # Validation at each check epoch batch
            if epoch % check_epochs != 0:
                continue

            model.eval() # set the model to evaluation form, disabling regularisation and training features

            with torch.no_grad(): #predictions performed with no gradient calculations
                y_pred_train = model(X_train)
                y_pred_val = model(X_val)

                t_rmse = loss_fn(y_pred_train, y_train)
                v_rmse = loss_fn(y_pred_val, y_val)

                print('Epoch %d : train RMSE  %.4f, val RMSE %.4f ' % (epoch, t_rmse, v_rmse), file=f)
                print('Epoch %d : train RMSE  %.4f, val RMSE %.4f ' % (epoch, t_rmse, v_rmse))
                
            ## Learning rate scheduler step
            scheduler.step(v_rmse)

            ## early stopping check to avoid overfitting
            early_stop(v_rmse, model)

            if early_stop.early_stop:
                print('Early stopping')
                break

    ## Load the last best model before training degrades         
    model.load_state_dict(torch.load('S2S_trained_model.pt'))


In [ ]:
training_S2S(model=model, optimizer=optimizer, loss_fn=loss_fn, loader=loader, scheduler=scheduler,
            num_epochs=num_epochs, check_epochs=check_epochs, pred_steps=pred_steps,
            training_prediction= 'mixed',tf_ratio=tf_ratio,dynamic_tf=dynamic_tf,
            X_val=X_val, y_val=y_val,
            saveas='S2S_out')

In [ ]:
# load final model back for train, validation visualization and later test prediction

# S2S instance
model = LSTM_S2S(input_size, hidden_size, output_size, pred_steps)

# load the final model
model.load_state_dict(torch.load('S2S_trained_model.pt'))

### Training and validation data sets visualization

In [ ]:
with torch.no_grad():
    try:
        y_pred_train = model.predict(X_train)
        y_pred_val = model.predict(X_val)
    except:
        y_pred_train = model(X_train)
        y_pred_val = model(X_val)
        
#     y_pred_test = model(X_test)
    print(y_pred_train.shape, y_pred_val.shape)

In [ ]:
num_features = input_size #Nd and IA
num_cases = len(train_cases)
colors = sns.color_palette("coolwarm", num_cases)

# Loop over features
for f_idx in range(num_features):
    plt.figure(figsize=(12,6))
    s_idx = -1
    
    for seq, case in zip(range(num_cases), train_cases):
        # Target plots, true data from CFD
        p = plt.plot(train[:, seq, f_idx], label=f'Target {str(case)}', color=colors[seq % len(colors)], linewidth = 2) # train has shape [times,cases,features]
        
        # Train predicted values
        if seq == 0:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[:train_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Train Predicted {str(case)}',linewidth = 4)
        else:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[train_casebatch[seq-1]:train_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Train Predicted {str(case)}', linewidth = 4)
            
    if f_idx == 1:
        plt.ylim(0.6, 1.1)


    plt.legend()
    plt.xlim(40, 105)
    plt.title(f'Training Prediction for {features[f_idx]}')
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[f_idx]}')

In [ ]:
num_features = input_size #Nd and IA
num_cases = len(val_cases)
colors = sns.color_palette("husl", num_cases)

# Loop over features
for f_idx in range(num_features):
    plt.figure(figsize=(12,6))
    s_idx = -1
    
    for seq, case in zip(range(num_cases), val_cases):
        # Target plots, true data from CFD
        p = plt.plot(train[:, seq, f_idx], label=f'Target {str(case)}', color=colors[seq % len(colors)],linewidth = 2) # train has shape [times,cases,features]
        
        # Train predicted values
        if seq == 0:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[:val_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Val Predicted {str(case)}',linewidth = 4)
        else:
            plt.plot(range(wid_size-1,len(train)),
                     y_pred_train[val_casebatch[seq-1]:val_casebatch[seq],s_idx,f_idx], 
                     c=p[0].get_color(),linestyle=':', label=f'Val Predicted {str(case)}',linewidth = 4)
    if f_idx == 1:
            plt.ylim(0.6, 1.1)


    plt.legend()
    plt.xlim(40, 105)
    plt.title(f'Validation prediction for {features[f_idx]}')
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[f_idx]}')

### Rollout function to carry out predictions in test data sets

In [ ]:
# Function to predict future values using rollout with the LSTM model
def rollout(model, input_seq, future_steps):
    
    ## setting to eval mode and dropping gradient calculation for prediction
    model.load_state_dict(torch.load('S2S_trained_model.pt'))
    model.eval()
    with torch.no_grad():
        
        reshaped_input = np.transpose(input_seq, (1,0,2)) #reshaping to case,inputdata,features

        gen_seq = torch.tensor(reshaped_input).clone().detach() #detaching tensor from computational graph so gradients are not required

        ### how many predicted windows will be calculated based on the input_steps
        num_forwards = int(future_steps / steps_out) + 1
        print(f'prediction iterates for {num_forwards} times.')

        ## window predicton
        for i in range(num_forwards):

            output = model(gen_seq) #LSTM pass

            gen_seq = torch.cat((torch.tensor(gen_seq), torch.tensor(output)), dim=1) # concatenates the predicted outputs to the initial input sequence
            # This new sequence is fed as the new input tensor and used for the next set of output predictions.               

    return gen_seq

In [ ]:
input_seq = test[:steps_in,:,:]
future_steps = test.shape[0] - steps_in
rollout_seq = rollout(model, input_seq, future_steps)
print(input_seq.shape)
print(rollout_seq.shape)

In [ ]:
from sklearn.metrics import r2_score

colors = sns.color_palette("cubehelix", len(test_cases))

for f_idx in range(input_size):
    plt.figure()
    for i, case in enumerate(test_cases):
        r2 = r2_score(test[:,i,f_idx],rollout_seq[i,:,f_idx][:test.shape[0]])

        p = plt.plot(test[:,i,f_idx], label=f'Target {case}, r2:{r2:.4f}',color = colors[i % len(colors)],linewidth = 2)
        plt.plot(rollout_seq[i,:,f_idx],c=p[0].get_color(), linestyle=':',label=r'Prediction',linewidth = 4)
    
    plt.legend()
    plt.title(f'Prediction for: {features[f_idx]}')
    plt.xlim(35, 105)
    plt.xlabel('Time steps')
    plt.ylabel(f'Scaled {features[f_idx]}')